In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
from kidney.datasets.kaggle import get_reader, SampleType
from kidney.utils.mask import rle_decode

In [ ]:
reader = get_reader()

In [ ]:
test_keys = reader.get_keys(SampleType.Unlabeled)

In [ ]:
test_keys

In [ ]:
train_keys = reader.get_keys(SampleType.Labeled)

In [ ]:
train_keys

In [ ]:
predicted_masks = pd.read_csv("/mnt/fast/data/kidney/outputs/best_f1_score.csv").set_index("id").to_dict("index")

In [ ]:
samples = {}
for i, key in enumerate(test_keys, 1):
    print(f"[{i}/{len(test_keys)}] {key}")
    sample = reader.fetch_one(key)
    h, w = sample["image"].shape[:2]
    encoded_mask = predicted_masks[key]["predicted"]
    decoded_mask = rle_decode(encoded_mask, (h, w))
    samples[key] = {"image": sample["image"], "prediction": decoded_mask}

In [ ]:
samples.keys()

In [ ]:
def mask_to_contours(mask: np.ndarray, take_nth: int):
    cnts, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnts = [cnt.squeeze(1)[::take_nth].tolist() for cnt in cnts]
    cnts = [cnt + [cnt[0]] for cnt in cnts]
    return cnts

In [ ]:
!ls /mnt/fast/data/supervisor/contours

In [ ]:
def make_id(n: int):
    import random
    domain = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
    return "".join([random.choice(domain) for i in range(n)])

In [ ]:
# import json, os
# output_dir = "/mnt/fast/data/supervisor/contours"
# # os.makedirs(output_dir, exist_ok=True)
# for key, sample in samples.items():
#     contours = mask_to_contours(sample["prediction"], take_nth=5)
#     contours = [{"id": make_id(20), "points": contour} for contour in contours]
#     with open(f"{output_dir}/{key}.json", "w") as fp:
#         json.dump(contours, fp)

In [ ]:
import json, os
input_dir = "/mnt/fast/data/supervisor/contours"
for fn in os.listdir(input_dir):
    for key in train_keys:
        if not fn.startswith(key):
            continue
        print(f"matched: {key}")
        path = os.path.join(input_dir, fn)
        with open(path, "r") as fp:
            contours = json.load(fp)
        contours = [{"id": make_id(20), "points": contour} for contour in contours]
        with open(path, "w") as fp:
            json.dump(contours, fp)
        break

In [ ]:
# contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# cnt = contours[0].squeeze()
# xs, ys = cnt[:, 0], cnt[:, 1]
# x_min, x_max = xs.min(), xs.max()
# y_min, y_max = ys.min(), ys.max()
# margin = 10
# x_min -= margin
# x_max += margin
# y_min -= margin
# y_max += margin
# cell = sample["image"][y_min:y_max, x_min:x_max]

# cnt[:, 0] -= x_min
# cnt[:, 1] -= y_min
# plt.imshow(sample["mask"][y_min:y_max, x_min:x_max], alpha=0.4)
# plt.imshow(cv.drawContours(cv.UMat(cell), [cnt], 0, (255, 0, 0), 3).get(), alpha=0.4)